# Hello Jupyter

In [3]:
from sklearn import neighbors, datasets
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
from sklearn import preprocessing
%matplotlib inline

## Load dataset samples from MILE dataset


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split

ds = MileDataset(full_dataset=True)
# ds = MileDataset(full_dataset=False)

X = ds.get_X()
y = ds.get_y()
print(X)
print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

classifier = KNeighborsClassifier(n_neighbors=4, algorithm="auto")
classifier = classifier.fit(X_train, y_train)

score = classifier.score(X_test, y_test)

print(score)
# print(y)
# for d in X:
#     print(len(d))